Open Play Shots - Expected Goal Model - Model Tuning and Building - GBM

In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import open_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [3]:
model_version = 1
model_name = 'expected_goal_open'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [4]:
RESPONSE = ModellingDataContract.RESPONSE_GOAL

Load Data

In [5]:
df_modelling = pd.read_csv(open_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
9211,211,kickIn,behind,1705.0,4,744,775.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,56.0,-36.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,1,0,1,108.0,33.0,75.0,Kick,Gather,Kick,Handball Received,56.0,48.0,7.0,4.0,-36.0,-44.0,-47.0,-37.0,775.0,771.0,769.0,766.0,31.0,4.0,11.313708,24.0,36.0,43.266615,0.982794,56.309932,0.082503,4.727050,0.258900,14.833859,576.0,3.178054,False,56,True
9212,212,kickIn,goal,1740.0,4,787,900.0,Geelong,Geelong,Brandan Parfitt,Brandan_Parfitt,Kick,75.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,6,0,109.0,34.0,75.0,Kick,Handball Received,Handball,Loose Ball Get,75.0,75.0,76.0,75.0,0.0,-1.0,-5.0,-6.0,900.0,899.0,898.0,897.0,113.0,1.0,1.000000,5.0,0.0,5.000000,0.000000,0.000000,2.168022,124.218492,-1.429422,-81.899864,25.0,1.609438,False,76,True
9213,218,possGain,goal,1837.0,4,1136,1198.0,Geelong,Geelong,Sam De Koning,Sam_De_Koning,Kick,78.0,1.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,6,0,115.0,40.0,75.0,Kick,Gather,Kick,Handball Received,78.0,78.0,65.0,64.0,1.0,1.0,-12.0,-9.0,1198.0,1193.0,1190.0,1188.0,62.0,5.0,0.000000,2.0,1.0,2.236068,0.463648,26.565051,-2.442748,-139.959156,-0.440569,-25.242748,4.0,0.693147,False,96,False
9214,227,throwIn,goal,1901.0,4,1452,1458.0,Geelong,Geelong,Joel Selwood,Joel_Selwood,Kick,37.0,17.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,6,0,121.0,46.0,75.0,Kick,Gather,Knock On,Handball,37.0,37.0,43.0,34.0,17.0,19.0,20.0,25.0,1458.0,1457.0,1455.0,1453.0,6.0,1.0,2.000000,43.0,17.0,46.238512,0.376490,21.571307,0.129338,7.410516,0.403551,23.121747,1849.0,3.761200,False,98,False
9215,233,possGain,goal,1971.0,4,1744,1753.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,48.0,-20.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,0,6,133.0,46.0,87.0,Kick,Bounce,Handball Received,Handball,48.0,38.0,35.0,37.0,-20.0,-31.0,-33.0,-37.0,1753.0,1752.0,1748.0,1747.0,9.0,1.0,14.866069,32.0,20.0,37.735925,0.558599,32.005383,0.144862,8.299977,0.461317,26.431498,1024.0,3.465736,False,53,True


In [6]:
training_data = df_modelling[df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]
test_data = df_modelling[~df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]

In [7]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]

In [8]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Behind,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
4,38,possGain,miss,392.0,1,1148,1153.0,Sydney,Sydney,Errol Gulden,Errol_Gulden,Kick,40.0,-11.0,clanger,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,0,0,18.0,9.0,9.0,Kick,Handball Received,Handball,Loose Ball Get,40.0,36.0,34.0,32.0,-11.0,-10.0,-9.0,-7.0,1153.0,1150.0,1149.0,1148.0,5.0,3.0,4.123106,38.0,11.0,39.560081,0.281772,16.144339,0.156423,8.962369,0.495369,28.382581,1444.0,3.637586,False,88,False
6,46,possGain,goal,476.0,1,1383,1389.0,Sydney,Sydney,Isaac Heeney,Isaac_Heeney,Kick,56.0,6.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,6,0,6,19.0,9.0,10.0,Kick,Handball Received,Handball,Handball Received,56.0,58.0,58.0,58.0,6.0,21.0,25.0,26.0,1389.0,1389.0,1388.0,1387.0,6.0,0.0,15.132746,22.0,6.0,22.803509,0.266252,15.255119,0.276208,15.825576,0.987285,56.567262,484.0,3.091042,False,96,False
15,123,centreBounce,goal,1086.0,3,128,143.0,Brisbane Lions,Brisbane Lions,Zac Bailey,Zac_Bailey,Kick,32.0,8.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,6,6,0,51.0,58.0,-7.0,Kick,Handball Received,Handball,Gather,32.0,22.0,26.0,26.0,8.0,18.0,20.0,20.0,143.0,140.0,139.0,139.0,15.0,3.0,14.142136,46.0,8.0,46.690470,0.172191,9.865807,0.135683,7.774075,0.423021,24.237313,2116.0,3.828641,False,94,False
17,152,possGain,goal,1351.0,3,1224,1238.0,Sydney,Sydney,Logan McDonald,Logan_McDonald,Kick,75.0,0.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,6,0,6,58.0,77.0,-19.0,Kick,Handball Received,Handball,Uncontested Mark,75.0,74.0,67.0,62.0,0.0,-1.0,-2.0,-2.0,1238.0,1238.0,1236.0,1235.0,14.0,0.0,1.414214,3.0,0.0,3.000000,0.000000,0.000000,-15.483871,-887.160457,-0.692641,-39.685388,9.0,1.098612,False,83,False
23,185,throwIn,miss,1611.0,4,279,315.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,37.0,21.0,clanger,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,0,0,65.0,107.0,-42.0,Kick,Loose Ball Get,Kick Inside 50 Result,Kick Into F50,37.0,37.0,41.0,-6.0,21.0,24.0,29.0,43.0,315.0,314.0,306.0,305.0,36.0,1.0,3.000000,41.0,21.0,46.065171,0.473356,27.121303,0.124257,7.119376,0.387814,22.220095,1681.0,3.713572,False,98,False


Preprocess Data

In [9]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [10]:
X_train_preproc = X_train[ModellingDataContract.modelling_feature_list]
X_test_preproc = X_test[ModellingDataContract.modelling_feature_list]

In [11]:
X_train_preproc.shape, X_test_preproc.shape

((7353, 2), (1863, 2))

In [12]:
X_train_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
0,56.080300,0.383818
1,47.010637,0.416065
2,36.878178,0.708626
3,46.486557,0.328553
5,59.682493,0.546167


In [13]:
X_test_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
4,39.560081,0.281772
6,22.803509,0.266252
15,46.690470,0.172191
17,3.000000,0.000000
23,46.065171,0.473356


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [14]:
monotone_cols = []
monotone_increasing = [1]*len(monotone_cols)
monotonicity_dict = dict(zip(monotone_cols, monotone_increasing))

In [15]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train)

In [16]:
xgb_tuner.training_data.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
0,56.080300,0.383818
1,47.010637,0.416065
2,36.878178,0.708626
3,46.486557,0.328553
5,59.682493,0.546167


In [17]:
xgb_tuner.tune_hyperparameters()

[I 2023-06-28 10:51:37,865] A new study created in memory with name: no-name-19f5e4b2-d979-4998-83b5-d5cff0dd4ec0
[I 2023-06-28 10:51:37,979] Trial 0 finished with value: 0.6176576789863509 and parameters: {'max_depth': 16, 'min_child_weight': 81, 'eta': 0.7036025097970883, 'gamma': 0.6778773216040263, 'lambda': 0.0050759203506917155, 'alpha': 0.0014974313070112934, 'subsample': 0.799479435408851, 'colsample_bytree': 0.4325298897858806}. Best is trial 0 with value: 0.6176576789863509.
[I 2023-06-28 10:51:37,995] Trial 1 finished with value: 0.6636183787633267 and parameters: {'max_depth': 10, 'min_child_weight': 58, 'eta': 0.05840414453570081, 'gamma': 0.0007061462977899657, 'lambda': 4.909719073373293, 'alpha': 7.79403504345474, 'subsample': 0.31601281996503544, 'colsample_bytree': 0.4337182470104103}. Best is trial 0 with value: 0.6176576789863509.
[I 2023-06-28 10:51:38,019] Trial 2 finished with value: 0.6527150336273534 and parameters: {'max_depth': 17, 'min_child_weight': 24, 'et

Number of finished trials:  100
Best trial:
  Value: 0.6121247645279893
  Params: 
    max_depth: 3
    min_child_weight: 58
    eta: 0.7220264986903343
    gamma: 0.0007737496851066291
    lambda: 0.008886739610856814
    alpha: 0.006577611565995355
    subsample: 0.7685041998190734
    colsample_bytree: 0.3438968214738812


In [18]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 3,
 'min_child_weight': 58,
 'eta': 0.7220264986903343,
 'gamma': 0.0007737496851066291,
 'lambda': 0.008886739610856814,
 'alpha': 0.006577611565995355,
 'subsample': 0.7685041998190734,
 'colsample_bytree': 0.3438968214738812}

Training Model - SuperXGBClassifier class for training and predictions

In [19]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = {}

In [20]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test, 
                               params = params)

In [21]:
super_xgb.fit()

[0]	validation_0-logloss:0.66381	validation_1-logloss:0.66827
[1]	validation_0-logloss:0.62835	validation_1-logloss:0.63352
[2]	validation_0-logloss:0.62521	validation_1-logloss:0.63159
[3]	validation_0-logloss:0.62051	validation_1-logloss:0.62776
[4]	validation_0-logloss:0.61898	validation_1-logloss:0.62771
[5]	validation_0-logloss:0.61783	validation_1-logloss:0.62783
[6]	validation_0-logloss:0.61785	validation_1-logloss:0.62827
[7]	validation_0-logloss:0.61753	validation_1-logloss:0.62946
[8]	validation_0-logloss:0.61678	validation_1-logloss:0.62933
[9]	validation_0-logloss:0.61628	validation_1-logloss:0.63219
[10]	validation_0-logloss:0.61561	validation_1-logloss:0.63197
[11]	validation_0-logloss:0.61529	validation_1-logloss:0.63202
[12]	validation_0-logloss:0.61494	validation_1-logloss:0.63198
[13]	validation_0-logloss:0.61475	validation_1-logloss:0.63153
[14]	validation_0-logloss:0.61460	validation_1-logloss:0.63205
[15]	validation_0-logloss:0.61433	validation_1-logloss:0.63116
[1

In [22]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.3438968214738812, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0007737496851066291, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.7220264986903343, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=58, missing=nan, monotone_constraints={},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [23]:
super_xgb.xgb_model.get_booster().feature_names

['Distance_to_Middle_Goal', 'Angle_to_Middle_Goal']

In [24]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [25]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]

Check Average Predictions

In [28]:
train_probas.mean(), training_data[RESPONSE].mean(), train_probas.mean(), test_data[RESPONSE].mean()

(0.42013296, 0.4180606555147559, 0.42013296, 0.42619431025228127)

Export model

In [29]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [30]:
train_info = training_data[['Match_ID', "Chain_Number", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Goal,Distance_to_Middle_Goal,Angle_to_Middle_Goal,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,1,1,56.080300,0.383818,0,0.269926
1,202101_BrisbaneLions_Sydney,6,1,47.010637,0.416065,0,0.363295
2,202101_BrisbaneLions_Sydney,10,1,36.878178,0.708626,0,0.325422
3,202101_BrisbaneLions_Sydney,21,1,46.486557,0.328553,0,0.324478
4,202101_BrisbaneLions_Sydney,43,0,59.682493,0.546167,0,0.305165


In [31]:
test_info = test_data[['Match_ID', "Chain_Number", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Goal,Distance_to_Middle_Goal,Angle_to_Middle_Goal,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,38,0,39.560081,0.281772,0,0.396657
1,202101_BrisbaneLions_Sydney,46,1,22.803509,0.266252,1,0.530800
2,202101_BrisbaneLions_Sydney,123,1,46.690470,0.172191,0,0.396657
3,202101_BrisbaneLions_Sydney,152,1,3.000000,0.000000,1,0.868129
4,202101_BrisbaneLions_Sydney,185,0,46.065171,0.473356,0,0.363295


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")